# Машинное обучение для текстов. Проект для «Викишоп»

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построить модель со значением метрики качества *F1* не меньше 0.75. 


## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re 
import datetime

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from catboost import Pool, CatBoostClassifier

from tqdm.notebook import tqdm

# метрики
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, confusion_matrix

import zlib
from scipy.sparse import coo_matrix, hstack

from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_score, \
                            recall_score, accuracy_score, precision_recall_curve

from termcolor import colored

import warnings
warnings.filterwarnings('ignore')

In [ ]:
random_magic = 1024
F1_target_score = 0.75

Выполним загрузку пакета nltk для символьной и статистической обработки естественного языка

In [ ]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

nltk.download('punkt')
nltk.download('stopwords')

stopwords = nltk_stopwords.words('english')

In [ ]:
set(stopwords)

Расширим список стоп-слов английского языка.

In [ ]:
stopwords.extend(['i\'m', 'hi', '\'m', '\'t', '\'s', '\'', 'u', 'ok'])

In [ ]:
RANDOM_MAGIC = 1024
F1_TARGET_SCORE = 0.75

Выполним загрузку данных.

In [ ]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
target = 'toxic'

Данные представлены таблицей с двумя столбцами. Столбец text содержит текст комментария, а toxic — целевой признак, всего 159571 комментариев.

In [ ]:
data.duplicated().sum()

In [ ]:
print('Доля токсичных коментариев: {0:.1%}'.format(data['toxic'].mean()))

Посмотрим на примеры отдельных комментариев отнесенных к категории токсчиных.

In [ ]:
data[data[target] == 1].head(15)

Во-первых следует обратить внимание на то, что некоторые комментарии написаны в UPPERCASE. Чтобы не потерять эту информацию при последующей обработке текста, добавим новый признак к набору данных.

In [ ]:
data['is_upper'] = data['text'].str.isupper()

In [ ]:
print('Доля токсичных коментариев среди текстов в UPPERCASE: {0:.1%}'.format(
    data[data['is_upper'] == 1]['toxic'].mean()))

Как видим, доля токсичных комментариев среди текстов в UPPERCASE существенно выше чем во всем наборе данных, это связано с тем, что токсичные комментарии пишут с сильным эмоциональным окрасом. Соответственно указанный признак целесообразно использовать при обучении модели.

Далее проведем очистку текстов от знаков препинания, токенизацию и последующую лемматизацию исходного текста. Лемматизированный текст в дальнейшем будет использоваться для построения TF-IDF матрицы.

In [ ]:
def clear_text(text):
    eng = re.sub(r'[^A-Za-z \']', ' ', text) 
    
    text_tokens = tokenizer.tokenize(eng)

    return " ".join([word for word in text_tokens if not word in stopwords])

Создадим функцию замены тегов nltk на теги wordnet.

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.download('wordnet')

In [ ]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

Создадим функцию для лемматизации с уточненными частями речи. Будем использовать WordNetLemmatizer.

In [ ]:
def pos_lemmatize(text):
    
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    
    wnl = WordNetLemmatizer()
    lemmatized_sentence = []
    
    for word, tag in wordnet_tagged: 
        if tag is None: 
            lemmatized_sentence.append(word) 
        else:         
            lemmatized_sentence.append(wnl.lemmatize(word, tag)) 
    lemmatized_sentence = " ".join(lemmatized_sentence)
    
    return lemmatized_sentence

In [ ]:
data['clear_text'] = data['text'].str.lower().apply(clear_text)

In [ ]:
data['lemm_text'] = data['clear_text'].apply(pos_lemmatize)

In [ ]:
data['word_count'] = data['lemm_text'].str.count(r'[ ]') + 1

Кроме того, дополним обучающие признаки синтетическим признаком zip_ratio - соотношением объема сжатого лемматизированного текста к исходному объему текста. В основе генерации указанного признака предположение о том, что для токсичных комментариев относительно нетоксичных комментариев характерна другая информативность (возможно, чаще используются повторения слов и фраз целиком, более частое употребление стоп-слов), соответственно по степени сжатия указанные тексты также будут отличаться.

In [ ]:
def calc_compressed_ratio(text):
    text_rep = text.replace(' ', '')
    text_len = len(text)

    if (text_len > 0):
        compressed = zlib.compress(bytes(text_rep, 'utf-8'))
        return len(compressed) 
    
    return 0

In [ ]:
data['src_len'] = data['text'].str.len()
data['lemm_len'] = data['lemm_text'].str.len()

In [ ]:
data['zip_len'] = data['lemm_text'].apply(calc_compressed_ratio)
data['zip_ratio'] = data[['zip_len','lemm_len']].min(axis=1) / data['src_len']

Рассмотрим особенности и распределение обучающих признаков по целевым классам

In [ ]:
def describe_column_category(column, title, df):
    """
    функция отображения сводной информации о категориальном признаке:
    выводит в текстовом виде униклаьные значения признака и график (гистограмму) 
    распределения по категориям: действующие и ушедшие клиенты
    
    """
    print('Признак', column, ':\n')
    print('Уникальные значения (процент):')
    print(df[column].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')
    
    print("\nДоля по целевому признаку (токсичные/все комментарии):")
    print((
            df.query('toxic == 1')[column].value_counts() / 
            df[column].value_counts()
        ).mul(100).round(1).astype(str) + '%'
    )
        
    fig = px.histogram(
        df, 
        x = column,  
        color = 'toxic',
        color_discrete_map={
                0: 'Green', 1: 'Red'
            },
        opacity = 0.7,
        title = title
    )

    fig.update_layout(xaxis_title=title, yaxis_title='Число комментариев')
    fig.show() 

In [ ]:
def describe_column_numeric(column, title, df):
    """
    функция отображения сводной информации о числовом признаке:
    выводит в текстовом виде основные параметры распределения значений признака и график
    (гистограмму и "ящик с усами")распределения по категориям:
    действующие и ушедшие клиенты
    
    """
    print('Признак', column, ':')
    
    print(df[column].describe())
    
    fig = px.histogram(
        df, 
        x = column, 
        marginal = 'box', 
        color = "toxic",
        color_discrete_map={
                0: 'Green', 1: 'Red'
            },
        opacity = 0.7,
        title = title
    )

    fig.update_layout(xaxis_title=title, yaxis_title='Число комментариев')
    fig.show()

Рассмотрим тексты, в которых после лемматизации осталось только 1 слово.

In [ ]:
data.query('word_count == 1').head(20)

In [ ]:
print('Доля текстов с 1 словом после лемматизации относительно всех данных: {0:.1%}'.format(
    data.query('word_count == 1').shape[0] / data.shape[0]))

Среди указанных текстов есть как ошибочные (например, содержащие только дату или IP-адрес), так и пустые (например, "No, it doesn´t" - в силу особенностей английского языка предложения, состоящие только из стоп-слов могут быть значимыми в общем контексте - например, как ответ на предыдущий комментарий).

Доля таких текстов - менее 1%. Учитывая то, что в дальшнейшем можно встроить в модель дополнительные проверки (например, проверка по словарю ненормативной лексики), то рассматривать тексты, состоящие из 1 слова (после лемматизации) в целом нецелесообразно, исключим их из рассмотрения.

In [ ]:
data = data.query('word_count > 1')

Рассмотрим распределение исходных текстов по длине текста.

In [ ]:
describe_column_numeric('src_len', 'Длина текста', data)

Посмотрим на "выбросы" по длине текста, в качестве возможной границы отсечения возьмем Q3+3*IQR

In [ ]:
q3_3iqr = data['src_len'].quantile(0.75) + 3 * (
    data['src_len'].quantile(0.75) - data['src_len'].quantile(0.25))
q3_3iqr

In [ ]:
data[data['src_len'] > q3_3iqr].head(10)

In [ ]:
print('Доля текстов длины более {0} символов относительно всех данных: {1:.1%}'.format(int(q3_3iqr),
    data.query('src_len > @q3_3iqr').shape[0] / data.shape[0]))

print('Доля токсичных коментариев в указанных текстах: {0:.1%}'.format(
    data[data['src_len'] > q3_3iqr]['toxic'].mean()))

Доля токсичных комментариев среди отобранных "длинных" текстов немного меньше их доли в общем объеме данных. В целом объем "длинных" текстов составляет ~4.4% исходной выборки.

У нас недостаточно сведений о деятельности Интернет-магазина «Викишоп», но с большой долей уверенности можно предположить, что разумное ограничение на длину комментария/описания товара, оставляемого пользователем, вполне допустимо (в целом, это нормальная практика, когда при вводе текста пользователь ограничен в количестве вводимых символов/слов). Исключим указанные "длинные" тексты из дальнейшего рассмотрения.

In [ ]:
data = data.query('src_len <= @q3_3iqr')

In [ ]:
describe_column_numeric('src_len', 'Длина текста', data)

Теперь форма распределения текстов по длине близка к распределению Пуассона.

Рассмотрим распределение исходных текстов по длине лемматизированного текста.

In [ ]:
describe_column_numeric('lemm_len', 'Длина лемматизированного текста', data)

Посмотрим на "выбросы" по длине текста после лемматизации, в качестве возможной границы отсечения возьмем Q3+3*IQR

In [ ]:
q3_3iqr = data['lemm_len'].quantile(0.75) + 3 * (
    data['lemm_len'].quantile(0.75) - data['lemm_len'].quantile(0.25))
q3_3iqr

In [ ]:
data[data['lemm_len'] > q3_3iqr].head(10)

In [ ]:
print('Доля текстов длины более {0} символов после лемматизации относительно всех данных: {1:.1%}'.format(int(q3_3iqr),
    data.query('lemm_len > @q3_3iqr').shape[0] / data.shape[0]))

print('Доля токсичных коментариев в указанных текстах: {0:.1%}'.format(
    data[data['lemm_len'] > q3_3iqr]['toxic'].mean()))

Доля токсичных комментариев среди отобранных "длинных" текстов немного меньше их доли в общем объеме данных. В целом объем "длинных" текстов составляет ~1.3% исходной выборки.

Учитывая то, что мы ввели ограничение на общую длину оставляемого комментария, цеелесообразно также рассмотреть и длины текстов после лемматизации (или их соотношение с исходной длиной). Наверно здесь требуются более глубокие лингвистические исследования или статистические сведения об особенностях текстов на английском языке, но было бы разумно предположить, что "обычные" комментарии (оставляемые пользователями) с точки зрения длины лемматизированных текстов отличаются, например, от сгенерированных ботами или выдержками из специальной литературы. В рамках построения настоящей модели мы исключим указанные "длинные" лемматизированные тексты из дальнейшего рассмотрения.

In [ ]:
data = data.query('lemm_len <= @q3_3iqr')

In [ ]:
describe_column_numeric('lemm_len', 'Длина лемматизированного текста', data)

Рассмотрим распределение исходных текстов по соотношению объема сжатого текста к исходному объему.

In [ ]:
describe_column_numeric('zip_ratio', 'Доля сжатия', data)

In [ ]:
print('Доля текстов с коэффициентом сжатия менее 0.2 относительно всех данных: {1:.1%}'.format(int(q3_3iqr),
    data.query('zip_ratio < 0.2').shape[0] / data.shape[0])
     )
print('Доля токсичных коментариев в указанных текстах: {0:.1%}'.format(
    data.query('zip_ratio < 0.2')['toxic'].mean())
     )

In [ ]:
data.query('zip_ratio < 0.2').head(10)

Среди текстов с высоким коэффициентом сжатия доля токсичных существенно выше, чем в среднем по набору данных. При этом указанные тексты вполне могут быть написаны человеком. Их целесообразно оставить для обучения (этот признак может оказаться значимым).

Посмотрим на "выбросы" в районе 1.

In [ ]:
print('Доля текстов с коэффициентом сжатия равным 1 относительно всех данных: {1:.1%}'.format(int(q3_3iqr),
    data.query('zip_ratio == 1').shape[0] / data.shape[0]))

print('Доля токсичных коментариев в указанных текстах: {0:.1%}'.format(
    data.query('zip_ratio == 1')['toxic'].mean()))

In [ ]:
data.query('zip_ratio == 1').head(15)

Указанные тексты в основном похожи на служебные сообщения (в том числе, перенаправления на другие комментарии/статьи). В них доля токсичных совпадает со средней по набору данных.

Указанные тексты исключим из дальнейшего рассмотрения.

In [ ]:
data = data.query('zip_ratio < 1')

Рассмотрим долю токсичных комментариев в UPPERCASE.

In [ ]:
describe_column_category('is_upper', 'Текст в UPPERCASE', data)

Доля токсичных комментариев в UPPERCASE намного выше, чем в среднем по набору данных. Указанный признак целесообразно использовать при обучении.

In [ ]:
print('После удаления выбросов в наборе данных осталось: {0} записей'.format(data.shape[0]))

Выделим в исходном наборе данных обучающую и тестовую выборки со стратификацией по целевому признаку.

In [ ]:
df_train, df_test = train_test_split(
    data, test_size=0.15, random_state=random_magic, stratify=data[target])

Наиболее часто употребляемые слова.

In [ ]:
def get_text_all_class(df_data, target_value):
    text = " "
    category = df_data[df_data[target] == target_value]
    for idx, row in tqdm(category.iterrows(), total=category.shape[0]):
        text += row['lemm_text'] + " "
    return text

In [ ]:
def plot_class(df_data, target_value, figsize=(7, 5)):
    plt.figure(figsize=figsize)
    category = get_text_all_class(df_data, target_value)
    wordcloud = WordCloud(max_font_size=40).generate(category)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

Отобразим в виде облака слов наиболее часто встречаемые слова в каждом из классов.

In [ ]:
plot_class(df_train, 1)

In [ ]:
plot_class(df_train, 0)

Следует отметить, что для класса токсичных комментариев ожидаемо наиболее часто употребляемыми являются нецензурные слова. Вместе с тем, отдельные общеупотребительные слова (например, think, one, page, article, make) являются частыми в обоих классах.

### Выводы:
Исходные данные представлены в виде одной таблицы с двумя признаками:

* текст (комментарий)
* признак токсичности (целевой признак)
В таблице содержится 159571 запись с размеченными комментариями. Дубликатов в исходном наборе данных нет. Доля токсичных комментариев в общем объеме данных - 10%. При обучении модели следует учитывать то, что целевые классы несбалансированы.

По условиям задачи моделирования требуется обучить модель, позволяющую классифицировать текст по признаку токсичности. Указанная модель предполагается к использованию Интернет-магазином «Викишоп» при оценке описаний товаров, оставляемых пользователями. Модель должна искать токсичные комментарии и отправлять их на модерацию.

Метрикой качества является F1, целевым значением - 0.75. При этом следует обращать внимание на значение метрик полнота и точность. С точки зрения бизнеса интерпретировать эти метрики можно следующим образом:

* полнота (recall) тем выше, чем меньше ложноотрицательных прогнозов дает модель - т.е. меньше количество пропущенных "токсичных" комментариев - указанная метрика содержит "репутационные" риски и издержки магазина (их сложно перевести непосредственно в денежное выражение, это скорее репутационные потери от наличия на сайте токсичных (в том числе, возможно, содержащих нецензурную лексику) комментариев
* точность (precision) тем выше, чем меньше ложноположительных прогнозов - т.е. ошибочно отправленных на модерацию нетоксчиных комментариев - указанная метрика имеет непосредственное денежное выражение, это стоимость модерации одного комментария (у.е./час). О бизнесе Интернет-магазина «Викишоп» дополнительных данных не представлено, но с высокой степенью вероятности следует считать метрику полноты более значимой в денежном выражении относительно точности (репутационные риски обычно намного выше чем стоимость оплаты труда оператора/модератора).

На этапе подготовки данных обучающие признаки дополнены несколькими синтетическими признаками, в частности zip_ratio - соотношением объема сжатого лемматизированного текста к исходному объему текста. В основе генерации указанного признака предположение о том, что для токсичных комментариев относительно нетоксичных комментариев характерна другая информативность (возможно, чаще используются повторения слов и фраз целиком, более частое употребление стоп-слов), соответственно по степени сжатия указанные тексты также будут отличаться.

На этапе анализа проведена оценка распределения значений отдельных признаков по целевым классам. Устранены отдельные выбросы, в том числе по длине представленных текстов, на основе предположения об имеющихся на сайте Интернет-магазина «Викишоп» разумных ограничениях на длину комментария/описания товара, оставляемого пользователем.

Данные подготовлены для последующего обучения моделей.

## Обучение моделей

Рассчитаем частоты слов обучающей выборки и на их основе составим матрицы TF-IDF для корпуса слов обучающей и тестовой выборки.

In [ ]:
corpus = df_train['lemm_text']

count_tf_idf = TfidfVectorizer() 

tf_idf_train = count_tf_idf.fit_transform(corpus) 

tf_idf_train

In [ ]:
corpus = df_test['lemm_text']

tf_idf_test = count_tf_idf.transform(corpus) 

tf_idf_test

In [ ]:
df_train_Y = df_train[target]
df_test_Y = df_test[target]

Дополним сформированные TF-IDF матрицы значениями дополнительных обучающих признаков.

In [ ]:
df_train_X = tf_idf_train
df_train_X = hstack((df_train_X, np.array(df_train['is_upper'])[:,None]))
df_train_X = hstack((df_train_X, np.array(df_train['zip_ratio'])[:,None]))
df_train_X = hstack((df_train_X, np.array(df_train['lemm_len'])[:,None]))

In [ ]:
df_test_X = tf_idf_test
df_test_X = hstack((df_test_X, np.array(df_test['is_upper'])[:,None]))
df_test_X = hstack((df_test_X, np.array(df_test['zip_ratio'])[:,None]))
df_test_X = hstack((df_test_X, np.array(df_test['lemm_len'])[:,None]))

In [ ]:
def print_roc_auc_plot(df_roc_auc, roc_auc_val, color):
    """
    функция печати графика ROC-кривой
    df_roc_auc - dataframe исходных данных для построения графика ROC-кривой(значения TPR, FPR и пороговые отсечки)
    roc_auc_val - значение метрики ROC-AUC
    color - цвет отображения
    
    """
    fig = px.area(
        df_roc_auc,
        x='fpr', y='tpr',
        title=f'ROC-кривая (AUC={roc_auc_val:.4f})',
        width=900, 
        height=500,
        color='color',
        color_discrete_map={
                'red': 'Red', 'green': 'Green'
            },
    )

    fig.add_trace(
        go.Scatter(
            x=[0, 1], 
            y=[0, 1], 
            name="Случайная модель",
            line=dict(color=color, width=2, dash='dash'),
        )
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=0.9)
    fig.update_xaxes(constrain='domain')
    fig.update_layout(xaxis_title='FPR (доля ложноположительных)', yaxis_title='TPR (доля истинно положительных)')
    fig.show() 

In [ ]:
def print_scores(model_fitted, X_test, Y_test):
    """
    функция печати метрик обученной модели
    
    """
    predictions = model_fitted.predict(X_test)
    
    probabilities_valid = model_fitted.predict_proba(X_test)
    probabilities_one_valid = probabilities_valid[:, 1]

    f1_val = f1_score(Y_test, predictions)
    roc_auc_val = roc_auc_score(Y_test, probabilities_one_valid)
    
    print(colored("\x1b[1mF1: {0}\x1b[0m".format(f1_val), 'red' if f1_val < F1_TARGET_SCORE else 'green'))
    print('AUC-ROC:', roc_auc_val)
    print('Precision:', precision_score(Y_test, predictions))
    print('Recall:', recall_score(Y_test, predictions))
    print('Accuracy:', accuracy_score(Y_test, predictions))
    
    fpr, tpr, thresholds = roc_curve(Y_test, probabilities_one_valid) 

    df_roc_auc = pd.DataFrame()
    df_roc_auc['fpr'] = fpr
    df_roc_auc['tpr'] = tpr
    df_roc_auc['thresholds'] = thresholds
    df_roc_auc['color'] = 'red'
        
    if f1_val >= F1_TARGET_SCORE:
        df_roc_auc['color'] = 'green'
        print_roc_auc_plot(df_roc_auc, roc_auc_val, 'green')
    else:
        print_roc_auc_plot(df_roc_auc, roc_auc_val, 'red')

In [ ]:
def get_proba_df(model_fitted, X_test, Y_test):
    """
    функция формирования датафрейма со значениями метрик precision, recall и f1 в диапазоне порогов
    
    """
    probabilities_valid = model_fitted.predict_proba(X_test)
    probabilities_one_valid = probabilities_valid[:, 1]

    scores = []
    for threshold in np.arange(0.1, 0.9, 0.01):
        predicted_valid = probabilities_one_valid > threshold
        precision = precision_score(Y_test, predicted_valid)
        recall = recall_score(Y_test, predicted_valid)
        f1 = f1_score(Y_test, predicted_valid)

        scores.append([threshold, precision, recall, f1])
        
    df_scores = pd.DataFrame(scores, columns=['threshold', 'precision', 'recall', 'f1'])
    return df_scores

In [ ]:
def print_proba(df_scores):
    
    """
    функция вывода графика метрик precision, recall и f1

    """
    fig = px.area(
        df_scores,
        x='threshold', 
        y='f1',
        width=900, 
        height=500,
        color_discrete_map={
                'red': 'Red', 'green': 'Green'
            },
    )

    fig.add_trace(
        go.Scatter(
            x=df_scores['threshold'], 
            y=df_scores['precision'], 
            name="Точность (precision)",
            line=dict(color='red', width=2),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df_scores['threshold'], 
            y=df_scores['recall'], 
            name="Полнота (recall)",
            line=dict(color='blue', width=2),
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=[0.1,0.9],
            y=[F1_TARGET_SCORE, F1_TARGET_SCORE], 
            name="Целевое значение<br>метрики F1",
            line=dict(color='magenta', width=3),
        )
    )
        
    fig.update_yaxes(scaleanchor="x", scaleratio=0.9)
    fig.update_xaxes(constrain='domain')
    
    fig.update_layout(
        xaxis=dict(
            #range=[0.1, 0.9],

        ),
        yaxis=dict(
            range=[0.6, 0.95],
        ),
        
        
        xaxis_title='Порог', 
        yaxis_title='Метрика'
    )
    fig.show()

In [ ]:
cv = StratifiedKFold(6, random_state=RANDOM_MAGIC, shuffle=True)

In [ ]:
scoring = make_scorer(roc_auc_score)

Проведем обучение и последующий анализ нескольких моделей на подготовленных данных.

**Логистическая регрессия**

In [ ]:
model_lr = LogisticRegression(random_state=random_magic, max_iter=5000, solver='liblinear', class_weight='balanced')

param_grid_lr = {
    'model__C': [100, 10, 1.0],
}

In [ ]:
params = param_grid_lr
model = model_lr

grid_search = GridSearchCV(
    Pipeline([('model', model)]), 
    param_grid=params, 
    cv=cv, 
    scoring=scoring
)    

grid_search.fit(df_train_X, df_train_Y)

Выведем график кривой ROC-AUC.

In [ ]:
print_scores(grid_search, df_test_X, df_test_Y)

Выведем график зависимости метрик от используемых порогов отнесения к целевым классам.

In [ ]:
df_scores = get_proba_df(grid_search, df_test_X, df_test_Y)
print_proba(df_scores)

На графике видно, что целевое значение метрики F1 достигается в достаточно широком диапазоне порогов - примерно в интервале от 0.49 до 0.59. Вместе с тем, одновременно необходимо оценивать значения метрик полноты и точности. Учитывая то, что ранее мы определили метрику полноты более значимой с точки зрения ее "стоимости" в бизнесе, к приемлемым значениям порогов следует отнести более узкий интервал вокруг точки пересечения метрик - (0.6, 0.7).

In [ ]:
df_scores.loc[(df_scores["threshold"] >= 0.6) & (df_scores["threshold"] <= 0.7)]

**LinearSVC**

In [ ]:
model_svc = LinearSVC(random_state=random_magic, max_iter=5000, class_weight='balanced')

In [ ]:
clf = CalibratedClassifierCV(model_svc, cv = cv) 
clf.fit(df_train_X, df_train_Y)

In [ ]:
print_scores(clf, df_test_X, df_test_Y)

Выведем график зависимости метрик от используемых порогов отнесения к целевым классам.

In [ ]:
df_scores = get_proba_df(clf, df_test_X, df_test_Y)
print_proba(df_scores)

На графике видно, что целевое значение метрики F1 также достигается в достаточно широком диапазоне порогов - примерно в интервале от 0.17 до 0.56. К приемлемым значениям порогов следует отнести более узкий интервал вокруг точки пересечения метрик - (0.23, 0.33).

In [ ]:
df_scores.loc[(df_scores["threshold"] >= 0.22) & (df_scores["threshold"] <= 0.32)]

**CatBoost**

Перейдем к рассмотрению модели, которая учитывает не только частоты слов в тексте, но и частоты различных синтаксических N-грамм (последовательностей лемматизированных слов).

Дополнительно разделим обучающую выборку на обучающую и валидационную (доля валидационной примерно соответствует доле тестовой выборки для финальной проверки модели).

In [ ]:
train, valid = train_test_split(df_train,
                                                test_size=0.15, 
                                                stratify=df_train['toxic'], 
                                                shuffle=True, random_state=random_magic)

In [ ]:
feature_names = ['is_upper', 'zip_ratio', 'lemm_len', 'lemm_text']

cat_features = ['is_upper']

text_features = ['lemm_text']

In [ ]:
model = CatBoostClassifier(
    cat_features=cat_features,
    text_features=text_features,
    verbose=50,
    eval_metric='AUC',
    task_type="CPU",
    iterations=1000,
    learning_rate=0.2,      
    
    text_processing = {
        "tokenizers" : [{
            "tokenizer_id" : "Space",
            "separator_type" : "ByDelimiter",
            "delimiter" : " "
        }],

        "dictionaries" : [{
            "dictionary_id" : "BiGram",
            "token_level_type": "Letter",
            "max_dictionary_size" : "150000",
            "occurrence_lower_bound" : "1",
            "gram_order" : "2"
        },{
            "dictionary_id" : "Trigram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "3"
        },{
            "dictionary_id" : "Fourgram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "4"
        },{
            "dictionary_id" : "Fivegram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "5"
        },{
            "dictionary_id" : "Sixgram",
            "max_dictionary_size" : "150000",
            "token_level_type": "Letter",
            "occurrence_lower_bound" : "1",
            "gram_order" : "6"
        }
        ],

        "feature_processing" : {
            "default" : [
                    {
                    "dictionaries_names" : ["BiGram", "Trigram", "Fourgram", "Fivegram", "Sixgram"],
                    "feature_calcers" : ["BoW"],
                    "tokenizers_names" : ["Space"]
                },
                    {
                "dictionaries_names" : ["BiGram", "Trigram", "Fourgram", "Fivegram", "Sixgram"],
                "feature_calcers" : ["NaiveBayes"],
                "tokenizers_names" : ["Space"]
            },{
                "dictionaries_names" : [ "BiGram", "Trigram", "Fourgram", "Fivegram", "Sixgram"],
                "feature_calcers" : ["BM25"],
                "tokenizers_names" : ["Space"]
            },
            ],
        }
    }
)

In [ ]:
model.fit(
    train[feature_names], train[target],
    eval_set=(valid[feature_names], valid[target]),
    plot=True
)

In [ ]:
df_test_X = df_test[feature_names]
df_test_Y = df_test[target]

In [ ]:
print_scores(model, df_test_X, df_test_Y)

Выведем график зависимости метрик от используемых порогов отнесения к целевым классам.

In [ ]:
df_scores = get_proba_df(model, df_test_X, df_test_Y)
print_proba(df_scores)

В отличие от рассмотренных выше моделей на графике видно, что целевое значение метрики F1 также достигается не только в более широком диапазоне порогов - примерно в интервале от 0.13 до 0.76 - но и само значение метрики F1 существенно выше. К приемлемым значениям порогов следует отнести более узкий интервал вокруг точки пересечения метрик - (0.24, 0.40).

In [ ]:
df_scores.loc[(df_scores["threshold"] >= 0.27) & (df_scores["threshold"] <= 0.40)]

## Анализ наилучшей модели

Проведем анализ результатов, полученных при использовании модели CatBoost.

Проведём оценку значимости обучающих признаков.

In [ ]:
def print_feature_importance(arr_importance, column_names):
    """
    функция печати графика значимости обучающих признаков
    
    """
    rel_feature_imp = np.abs(100 * (arr_importance / max(arr_importance)))
    
    rel_feature_df = pd.DataFrame(
        {
            'features' : list(column_names),
            'rel_importance' : rel_feature_imp
        }
    )

    rel_feature_df = rel_feature_df.sort_values('rel_importance', ascending=False)
    
    plt.figure(figsize=(20, 10))
    plt.yticks(fontsize=15)

    ax = sns.barplot(
        x = 'rel_importance', 
        y = 'features',
        data = rel_feature_df,
        palette = 'Accent_r'
    )

    plt.xlabel('Относительная значимость', fontsize=25)
    plt.ylabel('Признаки', fontsize=25)
    plt.show()

In [ ]:
df_importance = model.get_feature_importance(prettified=True)

print_feature_importance(df_importance['Importances'], df_importance['Feature Id'])

Ожидаемо наиболее значимым признаком оказался лемматизированный текст (и соответственно производное от него векторное представление текста), но вместе с тем и введенные дополнительные признаки имеют ненулевую значимость.

Проведём анализ остатков.

In [ ]:
f1_threshold = 0.38

In [ ]:
probabilities_valid = model.predict_proba(df_test_X)
probabilities_one_valid = probabilities_valid[:, 1]

scores = []

predicted_valid = probabilities_one_valid > f1_threshold
precision = precision_score(df_test_Y, predicted_valid)
recall = recall_score(df_test_Y, predicted_valid)
f1 = f1_score(df_test_Y, predicted_valid)

scores.append([f1_threshold, precision, recall, f1])
        
df_scores = pd.DataFrame(scores, columns=['threshold', 'precision', 'recall', 'f1'])
df_scores

Добавим к таргету тестовой выборки предсказанные значения.

In [ ]:
df_test_Y_res = pd.DataFrame(df_test_Y.copy())
df_test_Y_res['pred'] = pd.DataFrame(np.transpose(predicted_valid.astype(int)), index=df_test_Y_res.index)
df_test_Y_res.head(5)

Сфомируем датафреймы с ложноположительными и ложноотрицательными значениями.

In [ ]:
df_fp = df_test_Y_res.query('pred > toxic')
df_fn = df_test_Y_res.query('pred < toxic')

In [ ]:
pd.set_option('display.max_colwidth', 0)

Посмотрим на распределение ложноотрицательных значений по длине исходного текста.

In [ ]:
describe_column_numeric(
    'src_len', 'Длина лемматизированного текста', 
    data.loc[df_fn.index][['toxic', 'src_len', 'lemm_len', 'text', 'lemm_text']]
)

Распределение в целом повторяет распределение на обучающей выборке, но при этом правая граница межквартильного размаха смещена вправо, т.е. больше "длинных" текстов попало в межквартильный интервал ошибки.

In [ ]:
data.loc[df_fn.index].query('src_len < 250').sample(30)[['toxic', 'is_upper', 'src_len', 'text', 'lemm_text']]

По указанным примерам можно сделать следующие предположения о возможных направлениях улучшения модели:

* следует лучше отбирать значения в UPPERCASE (например, считать долю символов в UPPERCASE относительно всего текста)
* следует отдельно интерпретировать нецензурные слова с пропусками букв (например, cr@p, f*ing, WTF, Dl2000CK)
* возможно следует подключить проверку по словарю/наличию орфографических ошибок (опять-таки размечать долю слов с ошибками относительно всего текста)

In [ ]:
data.loc[df_fn.index].query('src_len > 400').sample(15)[['toxic', 'is_upper', 'src_len', 'text', 'lemm_text']]

На примерах "длинных" текстов также можно сделать предположение о возможных направлениях улучшения модели:
исключать тексты на политическую тематику (например, с упоминанием стран, национальностей, политических партий) либо давать им соответствующую разметку.

Также посмотрим на примеры ложноположительных ответов.

In [ ]:
data.loc[df_fp.index].query('src_len < 250').sample(15)[['toxic', 'is_upper', 'src_len', 'text', 'lemm_text']]

Примеры ложноположительных ответов вместе с тем говорят скорее об ошибках в разметке исходных данных - очевидно наличие комментариев, которые по содержанию скорее следует отнести к токсичным (что модель и сделала):

* ILL COME TO YOUR HOUSE ND MAKE U SORE BETWEEN UR LEGS
* A JEW? OR NOT A JEW?
* JULIE \n\nWOW THANKS FOR BLOCKING ME! hateyouevenmore
* moved to http://www.freearchive.org/wiki2/index.php/Tape_editing\nfor safety (the wikipedia idiots don't want to include this page)
* NO I HATE IT BECAUSE YOU DELETE EVERYTHING
* shut up man! \n\nSHUT UP I DID NOT DO ANYTHING Click Here for what I Wrote To You A Few Months Ago.

Указанные примеры говорят об имеющихся недостатках в разметке исходных данных, которые целесообразно в последующем устранить (что также должно сказаться на повышении точности модели).

## Выводы
Исследование проводилось в интересах Интернет-магазина «Викишоп», запускающего новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Цель проекта - обучить модель классифицировать комментарии на позитивные и негативные. На исследование предоставлен набор данных с разметкой о токсичности правок.

Исходные данные представлены в виде одной таблицы с двумя признаками:

* текст (комментарий)
* признак токсичности (целевой признак)

В таблице содержится порядка 160 тысяч записей с размеченными комментариями. Дубликатов в исходном наборе данных нет. Доля токсичных комментариев в общем объеме данных - 10%. При обучении модели следует учитывать то, что целевые классы несбалансированы.

По условиям задачи моделирования требуется обучить модель, позволяющую классифицировать текст по признаку токсичности. Модель должна искать токсичные комментарии и отправлять их на модерацию.

Метрикой качества является F1, целевым значением - 0.75. При этом следует обращать внимание на значение метрик полнота и точность. С точки зрения бизнеса интерпретировать эти метрики можно следующим образом:

* полнота (recall) тем выше, чем меньше ложноотрицательных прогнозов дает модель - т.е. меньше количество пропущенных "токсичных" комментариев - указанная метрика содержит "репутационные" риски и издержки магазина (их сложно перевести непосредственно в денежное выражение, это скорее репутационные потери от наличия на сайте токсичных (в том числе, возможно, содержащих нецензурную лексику) комментариев
* точность (precision) тем выше, чем меньше ложноположительных прогнозов - т.е. ошибочно отправленных на модерацию нетоксчиных комментариев - указанная метрика имеет непосредственное денежное выражение, это стоимость модерации одного комментария (у.е./час). О бизнесе Интернет-магазина «Викишоп» дополнительных данных не представлено, но с высокой степенью вероятности следует считать метрику полноты более значимой в денежном выражении относительно точности (репутационные риски обычно намного выше чем стоимость оплаты труда оператора/модератора).

На этапе подготовки данных обучающие признаки дополнены несколькими синтетическими признаками.

На этапе анализа проведена оценка распределения значений отдельных признаков по целевым классам. Устранены отдельные выбросы, в том числе по длине представленных текстов, на основе предположения об имеющихся на сайте Интернет-магазина «Викишоп» разумных ограничениях на длину комментария/описания товара, оставляемого пользователем.

Проведено обучение двух моделей на основе матрицы TF-IDF значимости слов лемматизированного текста (модели линейной регрессии и LinearSVC), а также модели CatBoostClassifier на основе векторного представления различных N-грамм лемматизированного текста.

По результатам обучения модели получены следующие результаты:

* все модели позволили достичь целевое значение метрики F1
* наилучшее значение метрики показала модель CatBoostClassifier - F1 = 0.79, Точность - 0.81, Полнота - 0.77, при пороговом значении отнесения к положительному классу равным 0.38

Анализ неверных предсказаний модели позволил определить возможные направления для ее улучшения:

* следует лучше отбирать значения в UPPERCASE (например, считать долю символов в UPPERCASE относительно всего текста)
* следует отдельно интерпретировать нецензурные слова с пропусками букв (например, cr@p, f*ing, WTF, Dl2000CK)
* возможно следует подключить проверку по словарю/наличию орфографических ошибок (опять-таки размечать долю слов с ошибками относительно всего текста)
* исключать тексты на политическую тематику (например, с упоминанием стран, национальностей, политических партий) либо давать им соответствующую разметку

Кроме того, выборочная оценка ложноположительных ответов говорит скорее об ошибках в разметке исходных данных - очевидно наличие комментариев, которые по содержанию скорее следует отнести к токсичным (что модель и сделала). Указанные недостатки в разметке исходных данных целесообразно в последующем устранить для повышении точности модели.